In [1]:
Query = "Professor Andrew ng stanford"
questions = ["List the prior and most recent courses taught?", "from which university did the professor get their phd from", "from which university did the professor get their Bachelor from?", "What are the professor's research interests?", "what is the professor's email", "where is the professor's office","What field is the professor involved in?"]
questions

['List the prior and most recent courses taught?',
 'from which university did the professor get their phd from',
 'from which university did the professor get their Bachelor from?',
 "What are the professor's research interests?",
 "what is the professor's email",
 "where is the professor's office",
 'What field is the professor involved in?']

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import os
import re
import trafilatura as tr
import pandas as pd
import wikipedia as wiki
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import torch.nn.functional as F


/Users/devgoyal/anaconda3/envs/research/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class DocumentEmbedder:
    def __init__(self, model_name='bert-large-uncased'):
        self.model_name = model_name
        self.tokenizer =AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def embed_doc(self, documents):
        # Initialize an empty list to store the document embeddings
        document_embeddings = []

        # Loop through each document and embed it
        for doc in documents:
            # Tokenize the document and convert it to tensors
            inputs = self.tokenizer(doc, return_tensors='pt', truncation=True, padding=True)
            
            # Forward pass through the BERT model
            with torch.no_grad():
                outputs = self.model(**inputs)
                # Extract the last layer embeddings for each token
                # embeddings = outputs.last_hidden_state
            
            # Calculate the mean of token embeddings to get document-level embedding
            doc_embedding = self.mean_pooling(outputs, inputs['attention_mask'])
            doc_embedding = F.normalize(doc_embedding)
            # Convert the PyTorch tensor to a NumPy array
            doc_embedding= doc_embedding.numpy()[0]
            document_embeddings.append(doc_embedding)

        return document_embeddings
    
    def encode_question(self, question, max_length=128):
        inputs = self.tokenizer(question, return_tensors='pt', truncation=True, padding=True)
            
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Extract the last layer embeddings for each token
            # embeddings = outputs.last_hidden_state
        
        # Calculate the mean of token embeddings to get document-level embedding
        question_embedding = self.mean_pooling(outputs, inputs['attention_mask'])
        question_embedding = F.normalize(question_embedding)
        # Convert the PyTorch tensor to a NumPy array
        question_embedding_np= question_embedding.numpy()[0]

        return question_embedding_np

bert = DocumentEmbedder('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
from serpapi import GoogleSearch

def get_query(query):
    api_key = ""
    with open("api_key.txt") as f:
        api_key = f.read()
    params = {
        "q": query,
        "engine": "google",
        "api_key": api_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results['organic_results']
    return organic_results

organic_results = get_query(Query)



In [5]:
def parse_results(link):
    downloaded = tr.fetch_url(link)
    text = tr.extract(downloaded, include_formatting=True, favor_recall=True,no_fallback=True)
    if text == None:
        return ''
    return text

def get_scholar_results(author_id):
    params = {
        'engine': 'google_scholar_author',
        'author_id': author_id,
        'api_key' : 'd5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f'
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

def sanitize_filename(filename):
    # Replace disallowed characters with underscores
    sanitized = re.sub(r'[\/:*?"<>|]', '_', filename)
    return sanitized

dataset = {'titles': [], 'text': []}

named_entity = ""
for result in organic_results:
    if 'wikipedia.org' in result['link']:
        with open("Documents/wiki.txt", 'w') as f:
            wiki_results = wiki.search(result['title'])
            page = wiki.page(wiki_results[0], auto_suggest=False)
            f.write(page.content)
        dataset['titles'].append("wikiresult")
        dataset['text'].append(page.content)
            
    else:
        text = parse_results(result['link'])
        if named_entity.lower() in text.lower():
            with open("Documents/" + sanitize_filename(result['title']) + '.txt', 'w') as f:
                f.write(parse_results(result['link']))
            dataset['titles'].append(result['title'])
            dataset['text'].append(parse_results(result['link']))

df = pd.DataFrame(dataset)
# df.to_csv('Dataset.csv', index=False, sep='\t')


In [6]:
# split the text into passages

def split_sections(text):
    ''' given a text split the text into sections where each section starts with some a certain number of hashtags'''

    sections = re.split(r'#+', text)
    sections = [section.strip() for section in sections if section.strip() != '']
    return sections

def clean_text(text):
    '''clean the text'''
    # Define a regular expression pattern to match punctuations and special characters except for hashtags and @ symbols
    pattern = r"[^\w\s#@.]"

    text = text.lower()
    
    # Use regex to replace the pattern with an empty string, effectively removing punctuations and special characters
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def split_wikiresult(text):
    pattern = r'\n\n'
    passages = re.split(pattern, text)
    
    # Remove empty strings from the list of passages
    passages = [passage.strip() for passage in passages if passage.strip()]
    print(len(passages),passages)

    return passages



def split_documents(documents: dict) -> dict:
    """Split documents into passages"""
    titles, texts = [], []
    for title, text in zip(documents["titles"], documents["text"]):
        if text is not None:
            text = clean_text(text)
            passages = split_sections(text)
            if title == "wikiresult":
                # print(text)
                passages = split_wikiresult(text)
            for passage in passages:
                titles.append(title if title is not None else "")
                texts.append(passage)
    return {"title": titles, "text": texts}

documents = df.to_dict('list')
documents = split_documents(documents)


17 ['andrew yantak ng chinese 吳恩達 born 1976 is a britishamerican computer scientist and technology entrepreneur focusing on machine learning and artificial intelligence ai. ng was a cofounder and head of google brain and was the former chief scientist at baidu building the companys artificial intelligence group into a team of several thousand people.ng is an adjunct professor at stanford university formerly associate professor and director of its stanford ai lab or sail. ng has also made substantial contributions to the field of online education as the cofounder of both coursera and deeplearning.ai. he has spearheaded many efforts to democratize deep learning teaching over 2.5 million students through his online courses. he is one of the worlds most famous and influential computer scientists being named one of time magazines 100 most influential people in 2012 and fast companys most creative people in 2014. in 2018 he launched and currently heads the ai fund initially a 175million inve

In [7]:
# 
documents["embedding"] = bert.embed_doc(documents["text"])
print(len(documents["embedding"]))


21


In [8]:
new_df_embed = pd.DataFrame(documents)
print(new_df_embed.shape)

(21, 3)


In [9]:
label_to_id = {label: i for i, label in enumerate(new_df_embed['title'].unique())}
print(label_to_id)
id_to_label = {i: label for label, i in label_to_id.items()}

{'Andrew Ng | Stanford HAI': 0, 'Andrew Ng: Home': 1, 'wikiresult': 2, 'Andrew Ng, Instructor': 3, 'Andrew Ng': 4}


In [10]:
new_df_embed.head(-1)

,title,text,embedding
0,Andrew Ng | Stanford HAI,andrew ng\nfounder of deeplearning.ai and adju...,"[-0.09152999, -0.100887045, -0.044136543, -0.0..."
1,Andrew Ng: Home,origins of the modern mooc xmooc\nonline educa...,"[0.009042408, -0.11746689, -0.050064716, -0.02..."
2,wikiresult,andrew yantak ng chinese 吳恩達 born 1976 is a br...,"[-0.065571524, -0.08098844, -0.029409274, 0.02..."
3,wikiresult,biography \nng was born in the united kingdom ...,"[-0.12358991, -0.033212356, -0.004762583, 0.03..."
4,wikiresult,career,"[-0.05466407, 0.07951243, -0.01820488, 0.05403..."
5,wikiresult,academia and teaching \nng is a professor at s...,"[-0.1570711, -0.07947984, -0.036384065, 0.0457..."
6,wikiresult,industry \nfrom 2011 to 2012 he worked at goog...,"[-0.055602513, -0.105987914, -0.015087452, 0.0..."
7,wikiresult,research \nng researches primarily in machine ...,"[-0.11521618, -0.076109245, -0.03567165, -0.01..."
8,wikiresult,online education massive open online course \n...,"[-0.05723257, -0.11592515, -0.08311999, 0.0355..."
9,wikiresult,founding of coursera \nng started the stanford...,"[-0.11311955, -0.093213454, -0.027751502, -0.0..."


In [11]:
train_features = np.array(new_df_embed['embedding'].tolist())
train_labels = np.array([i for i in range(0, len(new_df_embed))])
print("train_features shape = ", train_features.shape)
print("train_labels shape = ", train_labels.shape)

train_features shape =  (21, 384)
train_labels shape =  (21,)


In [12]:

# Create a KNN classifier with cosine similarity as the metric
knn = KNeighborsClassifier(metric='cosine')

# Fit the KNN model with document embeddings
knn.fit(train_features, train_labels) 

KNeighborsClassifier(metric='cosine')

In [13]:
question_embedding = [bert.encode_question(question) for question in questions]

In [14]:
k = 5
top_k = knn.predict_proba(question_embedding)
contexts = []
for query in top_k:
    top_k_indices = np.argsort(query)[::-1][:k]
    conatenated_context = '\n\n'.join(new_df_embed['text'].iloc[top_k_indices])
    contexts.append(conatenated_context)
    print(conatenated_context)
    print('-------------------')


postcoursera work 
in 2019 ng launched a new course ai for everyone. this is a nontechnical course designed to help people understand ais impact on society and its benefits and costs for companies as well as how they can navigate through this technological revolution.

founding of coursera 
ng started the stanford engineering everywhere see program which in 2008 published a number of stanford courses online for free. ng taught one of these courses machine learning which includes his video lectures along with the student materials used in the stanford cs229 class. it offered a similar experience to mit opencourseware except it aimed at providing a more complete course experience equipped with lectures course materials problems and solutions etc. the see videos were viewed by the millions and inspired ng to develop and iterate new versions of online tech.within stanford they include daphne koller with her blended learning experiences and codesigning a peergrading system john mitchell cou

In [15]:
def get_answer(question, context, model_flan, tokenizer):
   
    input_text = "Answer the question as accurately and elaborate as possible based on the context \n\n context: v1 \n\n question: v2" 
    input_text = input_text.replace('v1', context)
    input_text = input_text.replace('v2', question)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model_flan.generate(input_ids,max_length=1024)
    return tokenizer.decode(outputs[0])


# get the answer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_flan = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
for i in range(len(questions)):
    print("Question: ", questions[i])
    # print("Context: ", contexts[i])
    print(get_answer(questions[i], contexts[i], model_flan, tokenizer))


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 512). Running this sequence through the model will result in indexing errors


Question:  List the prior and most recent courses taught?
<pad> cs229a</s>
Question:  from which university did the professor get their phd from
<pad> university of california berkeley</s>
Question:  from which university did the professor get their Bachelor from?
<pad> carnegie mellon university</s>
Question:  What are the professor's research interests?
<pad> deep learning and its applications to computer vision and speech including such applications as autonomous driving</s>
Question:  what is the professor's email
<pad> dr.ng</s>
Question:  where is the professor's office
<pad> stanford university</s>
Question:  What field is the professor involved in?
<pad> deep learning</s>


In [16]:
# clean the documents folder
for file in os.listdir('Documents'):
    os.remove('Documents/' + file)
